У цьому ДЗ ми потренуємось розв'язувати задачу багатокласової класифікації за допомогою логістичної регресії з використанням стратегій One-vs-Rest та One-vs-One, оцінити якість моделей та порівняти стратегії.

### Опис задачі і даних

**Контекст**

В цьому ДЗ ми працюємо з даними про сегментацію клієнтів.

Сегментація клієнтів – це практика поділу бази клієнтів на групи індивідів, які схожі між собою за певними критеріями, що мають значення для маркетингу, такими як вік, стать, інтереси та звички у витратах.

Компанії, які використовують сегментацію клієнтів, виходять з того, що кожен клієнт є унікальним і що їхні маркетингові зусилля будуть більш ефективними, якщо вони орієнтуватимуться на конкретні, менші групи зі зверненнями, які ці споживачі вважатимуть доречними та які спонукатимуть їх до купівлі. Компанії також сподіваються отримати глибше розуміння уподобань та потреб своїх клієнтів з метою виявлення того, що кожен сегмент цінує найбільше, щоб точніше адаптувати маркетингові матеріали до цього сегменту.

**Зміст**.

Автомобільна компанія планує вийти на нові ринки зі своїми існуючими продуктами (P1, P2, P3, P4 і P5). Після інтенсивного маркетингового дослідження вони дійшли висновку, що поведінка нового ринку схожа на їхній існуючий ринок.

На своєму існуючому ринку команда з продажу класифікувала всіх клієнтів на 4 сегменти (A, B, C, D). Потім вони здійснювали сегментовані звернення та комунікацію з різними сегментами клієнтів. Ця стратегія працювала для них надзвичайно добре. Вони планують використати ту саму стратегію на нових ринках і визначили 2627 нових потенційних клієнтів.

Ви маєте допомогти менеджеру передбачити правильну групу для нових клієнтів.

В цьому ДЗ використовуємо дані `customer_segmentation_train.csv`[скачати дані](https://drive.google.com/file/d/1VU1y2EwaHkVfr5RZ1U4MPWjeflAusK3w/view?usp=sharing). Це `train.csv`з цього [змагання](https://www.kaggle.com/datasets/abisheksudarshan/customer-segmentation/data?select=train.csv)

**Завдання 1.** Завантажте та підготуйте датасет до аналізу. Виконайте обробку пропущених значень та необхідне кодування категоріальних ознак. Розбийте на тренувальну і тестувальну вибірку, де в тесті 20%. Памʼятаємо, що весь препроцесинг ліпше все ж тренувати на тренувальній вибірці і на тестувальній лише використовувати вже натреновані трансформери.
Але в даному випадку оскільки значень в категоріях небагато, можна зробити обробку і на оригінальних даних, а потім розбити - це простіше. Можна також реалізувати процесинг і тренування моделі з пайплайнами. Обирайте як вам зручніше.

In [45]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from imblearn.over_sampling import SMOTENC
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
customer_segmentation_train_df = pd.read_csv("drive/MyDrive/1. ML/Модуль 2/2.4.Незбалансована багатокласова класифікація/customer_segmentation_train.csv")

In [4]:
customer_segmentation_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               8068 non-null   int64  
 1   Gender           8068 non-null   object 
 2   Ever_Married     7928 non-null   object 
 3   Age              8068 non-null   int64  
 4   Graduated        7990 non-null   object 
 5   Profession       7944 non-null   object 
 6   Work_Experience  7239 non-null   float64
 7   Spending_Score   8068 non-null   object 
 8   Family_Size      7733 non-null   float64
 9   Var_1            7992 non-null   object 
 10  Segmentation     8068 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 693.5+ KB


In [5]:
customer_segmentation_train_df.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


In [16]:
customer_segmentation_train_df.drop(columns=['ID'], inplace=True, errors='ignore')

In [25]:
print(customer_segmentation_train_df.columns)

Index(['Gender', 'Ever_Married', 'Age', 'Graduated', 'Profession',
       'Work_Experience', 'Spending_Score', 'Family_Size', 'Var_1',
       'Segmentation'],
      dtype='object')


In [42]:

#Видаляємо рядки з пропусками
customer_segmentation_train_df = customer_segmentation_train_df.dropna()

#Визначаємо таргет і фічі
target = 'Segmentation'
X = customer_segmentation_train_df.drop(columns=[target])
y = customer_segmentation_train_df[target]

# Розбиваємо на train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Визначаємо категоріальні і числові ознаки
categorical_features = X_train.select_dtypes(include=['object']).columns
num_features = X_train.select_dtypes(exclude=['object']).columns

# Для OneHot-кодування виключаємо 'Spending_Score', для якого буде OrdinalEncoder
ohe_cols = np.setdiff1d(categorical_features, ['Spending_Score'])

#Ініціалізуємо енкодери
ohe_encoder = OneHotEncoder(sparse_output=False, drop='if_binary')
ord_encoder = OrdinalEncoder(categories=[['Low', 'Average', 'High']])

# Кодуємо категоріальні ознаки
ohe_encoded_features_train = ohe_encoder.fit_transform(X_train[ohe_cols])
ohe_encoded_features_test = ohe_encoder.transform(X_test[ohe_cols])

ord_encoded_train = ord_encoder.fit_transform(X_train[['Spending_Score']])
ord_encoded_test = ord_encoder.transform(X_test[['Spending_Score']])

# Створюємо датафрейми з кодуваннями, з правильними колонками і індексами
ohe_encoded_train_df = pd.DataFrame(ohe_encoded_features_train,
                                    columns=ohe_encoder.get_feature_names_out(ohe_cols),
                                    index=X_train.index)

ohe_encoded_test_df = pd.DataFrame(ohe_encoded_features_test,
                                   columns=ohe_encoder.get_feature_names_out(ohe_cols),
                                   index=X_test.index)

ord_encoded_train_df = pd.DataFrame(ord_encoded_train,
                                    columns=['Spending_Score_Encoded'],
                                    index=X_train.index)

ord_encoded_test_df = pd.DataFrame(ord_encoded_test,
                                   columns=['Spending_Score_Encoded'],
                                   index=X_test.index)

# Об’єднуємо числові фічі з закодованими категоріальними
work_features = num_features.tolist() + list(ohe_encoded_train_df.columns) + ['Spending_Score_Encoded']

X_train_final = pd.concat([X_train[num_features], ohe_encoded_train_df, ord_encoded_train_df], axis=1)[work_features]
X_test_final = pd.concat([X_test[num_features], ohe_encoded_test_df, ord_encoded_test_df], axis=1)[work_features]

print("Train shape:", X_train_final.shape)
print("Test shape:", X_test_final.shape)

# Тепер X_train_final і X_test_final готові для моделювання без NaN


Train shape: (5332, 23)
Test shape: (1333, 23)


In [43]:
X_train_final

,Age,Work_Experience,Family_Size,Ever_Married_Yes,Gender_Male,Graduated_Yes,Profession_Artist,Profession_Doctor,Profession_Engineer,Profession_Entertainment,...,Profession_Lawyer,Profession_Marketing,Var_1_Cat_1,Var_1_Cat_2,Var_1_Cat_3,Var_1_Cat_4,Var_1_Cat_5,Var_1_Cat_6,Var_1_Cat_7,Spending_Score_Encoded
463,42,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
6977,49,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5545,41,0.0,2.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
144,43,0.0,5.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3491,48,0.0,2.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4557,62,0.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0
6266,48,3.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
6309,63,0.0,4.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
6505,57,0.0,2.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


**Завдання 2. Важливо уважно прочитати все формулювання цього завдання до кінця!**

Застосуйте методи ресемплингу даних SMOTE та SMOTE-Tomek з бібліотеки imbalanced-learn до тренувальної вибірки. В результаті у Вас має вийти 2 тренувальних набори: з апсемплингом зі SMOTE, та з ресамплингом з SMOTE-Tomek.

Увага! В нашому наборі даних є як категоріальні дані, так і звичайні числові. Базовий SMOTE не буде правильно працювати з категоріальними даними, але є його модифікація, яка буде. Тому в цього завдання є 2 виконання

  1. Застосувати SMOTE базовий лише на НЕкатегоріальних ознаках.

  2. Переглянути інформацію про метод [SMOTENC](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTENC.html#imblearn.over_sampling.SMOTENC) і використати цей метод в цій задачі. За цей спосіб буде +3 бали за це завдання і він рекомендований для виконання.

  **Підказка**: аби скористатись SMOTENC треба створити змінну, яка містить індекси ознак, які є категоріальними (їх номер серед колонок) і передати при ініціації екземпляра класу `SMOTENC(..., categorical_features=cat_feature_indeces)`.
  
  Ви також можете розглянути варіант використання варіації SMOTE, який працює ЛИШЕ з категоріальними ознаками [SMOTEN](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTEN.html)

In [47]:
# SMOTE тільки по числовим ознакам
smote_basic = SMOTE(random_state=42)

# Використовуємо тільки числові фічі для SMOTE
X_train_num = X_train_final[num_features]

X_train_smote_basic, y_train_smote_basic = smote_basic.fit_resample(X_train_num, y_train)

print("SMOTE Basic - до:", X_train_num.shape, "після:", X_train_smote_basic.shape)

SMOTE Basic - до: (5332, 3) після: (5548, 3)


In [35]:
# Знаходимо індекси категоріальних колонок в X_train_final
cat_feature_indeces = [X_train_final.columns.get_loc(col) for col in ohe_encoded_train_df.columns] + [X_train_final.columns.get_loc('Spending_Score_Encoded')]

# Ініціалізуємо SMOTENC з цими індексами
smote_nc = SMOTENC(categorical_features=cat_feature_indeces, random_state=42)

X_train_smote_nc, y_train_smote_nc = smote_nc.fit_resample(X_train_final, y_train)

print("SMOTENC - до:", X_train_final.shape, "після:", X_train_smote_nc.shape)


SMOTENC - до: (5332, 23) після: (5548, 23)


In [48]:
# Застосовуємо SMOTE-Tomek на повний набір ознак (числові + one-hot + ordinal)
smote_tomek = SMOTETomek(random_state=42)

X_train_smote_tomek, y_train_smote_tomek = smote_tomek.fit_resample(X_train_final, y_train)

print("SMOTE-Tomek - до:", X_train_final.shape, "після:", X_train_smote_tomek.shape)


SMOTE-Tomek - до: (5332, 23) після: (4726, 23)


**Завдання 3**.
  1. Навчіть модель логістичної регресії з використанням стратегії One-vs-Rest з логістичною регресією на оригінальних даних, збалансованих з SMOTE, збалансованих з Smote-Tomek.  
  2. Виміряйте якість кожної з натренованих моделей використовуючи `sklearn.metrics.classification_report`.
  3. Напишіть, яку метрику ви обрали для порівняння моделей.
  4. Яка модель найкраща?
  5. Якщо немає суттєвої різниці між моделями - напишіть свою гіпотезу, чому?

In [49]:
def train_evaluate_ovr(X_train, y_train, X_test, y_test):
    ovr_model = OneVsRestClassifier(LogisticRegression(solver='liblinear', max_iter=1000, random_state=42))
    ovr_model.fit(X_train, y_train)

    y_pred_train = ovr_model.predict(X_train)
    y_pred_test = ovr_model.predict(X_test)

    print("Classification report on TRAIN set:")
    print(classification_report(y_train, y_pred_train))
    print("Classification report on TEST set:")
    print(classification_report(y_test, y_pred_test))


In [50]:
# На оригінальних (оброблених) даних
train_evaluate_ovr(X_train_final, y_train, X_test_final, y_test)

# На SMOTENC (balanced) даних
train_evaluate_ovr(X_train_smote_nc, y_train_smote_nc, X_test_final, y_test)

# На SMOTE-Tomek balanced даних
train_evaluate_ovr(X_train_smote_tomek, y_train_smote_tomek, X_test_final, y_test)


Classification report on TRAIN set:
              precision    recall  f1-score   support

           A       0.43      0.50      0.47      1301
           B       0.41      0.13      0.20      1270
           C       0.49      0.69      0.57      1374
           D       0.65      0.69      0.67      1387

    accuracy                           0.51      5332
   macro avg       0.50      0.50      0.48      5332
weighted avg       0.50      0.51      0.48      5332

Classification report on TEST set:
              precision    recall  f1-score   support

           A       0.42      0.48      0.45       315
           B       0.45      0.15      0.22       302
           C       0.49      0.68      0.57       346
           D       0.66      0.69      0.67       370

    accuracy                           0.52      1333
   macro avg       0.50      0.50      0.48      1333
weighted avg       0.51      0.52      0.49      1333

Classification report on TRAIN set:
              precision

Якщо дивитись за f1-macro на тесті — трохи краща модель зі SMOTE (0.49), але різниця мінімальна.